# Algorithm 3 (General Attribute Exploration, Weaker Version)

In [ ]:
import pandas as pd
import math 

In [3]:
context_df = pd.read_csv('example.csv',sep=';')

In [4]:
context = (context_df.values.tolist(),context_df.shape)

In [5]:
class Implication:
    def __init__(self,p,c):
        self.premise = p.copy()
        self.consequent = c.copy()
        pass
    def __str__(self):
        return str(self.premise)+'->'+str(self.consequent)
    def __repr__(self):
        return str(self.premise)+'->'+str(self.consequent)
    def print_for_context(self, cont):
        old_values = range(0, len(cont))
        p_new = list(map(dict(zip(old_values, cont)).get, self.premise))
        c_new = list(map(dict(zip(old_values, cont)).get, self.consequent))
        return str(p_new)+'->'+str(c_new)

In [6]:
def Galois(X,context,t):
    if len(X)==0:
        if t=='ObjectsToAttributes': return set(range(0, context[1][1]))
        if t=='AttributesToObjects': return set(range(0, context[1][0]))
    if t=='ObjectsToAttributes':
        context_temp=[context[0][x] for x in list(X)]
    if t=='AttributesToObjects':
        transposed=list(zip(*context[0]))
        context_temp=[transposed[x] for x in list(X)]
    at=[sum(i) for i in zip(*context_temp)]
    attributes=set()
    for i in range(0,len(at)):
        if at[i]==len(X):   
            attributes.update(set([i]))
    return attributes

In [7]:
def Closure(A,L):
    Li=L.copy()
    while True:
        #print('Closure Start')
        stable=True   
        Lj=Li.copy()
        for i in Lj:
            if (i.premise.issubset(A)):
                A.update(i.consequent)
                stable=False
                Li.difference_update(set([i]))
        if stable==True:
            break
    return A

In [8]:
def NextClosure(A,M,K,L):
    for m in list(reversed(M)):
        if m in A:
            A.difference_update(set([m]))
        else:           
            A_temp=A.copy()
            A_temp.update(set([m]))
            B=Closure(A_temp,K)
            B_diff=B.copy()
            B_diff.difference_update(A)  
            B_L=B.copy()
            if all(i>=m for i in B_diff) and (B != Closure(B_L, L)):
                return B
    return -1

In [9]:
def expert_p(imp):
    AI =  Galois(imp.premise, context, 'AttributesToObjects')
    BI = Galois(imp.consequent, context, 'AttributesToObjects')
    # Является ли AI подмножеством BI
    if (AI.issubset(BI)) :
        return False
    else :
        # Контрпример
        counterexample = [0 for i in range(0, len(M))]
        for i in imp.premise :
            counterexample[i] = 1
        return counterexample

In [10]:
def col_num_to_name(col_nums, columns):
    old_values = range(0, len(columns))
    new = list(map(dict(zip(old_values, columns)).get, col_nums))
    return new

def example_to_column_names(example, columns):
    new = [columns[i] for i in range(0, len(example)) if example[i]==1]
    return new

def example_to_column_nums(example):
    new = [i for i in range(0, len(example)) if example[i]==1]
    return new

In [11]:
def ChooseQ(M, Pi_next, L, Ki) :
    Q = Pi_next.copy()
    return Closure(Q, L)  

In [12]:
def DeleteFalseImplications(L, C) :
    ex = example_to_column_nums(C)
    L_updated = L.copy()
    for i in L :
        if not (not i.premise.issubset(ex) or i.consequent.issubset(ex)) :
            L_updated.remove(i)
    return L_updated

In [13]:
def ExploringFaultyData(K, L, M):
    print('Значение M - ' + str(M))
    print('Значение K - ' + str(K))
    print('Значение L - ' + str(L))
    # i
    i = 0
    Pi = Closure(set(), K)
    print('Значение Pi - ' + str(Pi))
    Ki = K
    Li = L
    context_Li = []
    print('Контекст Li - ' + str(context_Li))

    while (True):
        # ii
        Pi=NextClosure(Pi, M, Ki, Li)
        print('NextClosure - ' + str(Pi))

        if Pi==-1:
            break
        
        # iii Выбор Q
        Qi_next = ChooseQ(M, Pi, Li, Ki)
        print('Выбранное Q - ' + str(Qi_next))

        # Инициализация импликации
        imp = Implication(Pi, Qi_next)
        print('Есть ли контрпример для ' + imp.print_for_context(context_df.columns))
        # Запрос к эксперту
        counterexample = expert_p(imp)
        print(counterexample)

        # iv - v
        if (counterexample) :
            # Удаление импликаций, противоречащих контрпримеру
            Li = DeleteFalseImplications(Li, counterexample)
            # Добавлен контрпример
            context_Li.append(counterexample)
            print('Контекст Li - ' + str(context_Li))
            print('Значение Li - ' + str(Li))
        else :
            Ki.add(imp)
            print(Ki)
        # vi
        i = i+1
    print(Ki)
    print(Li)

In [14]:
M=list(range(0,context[1][1]))
L = {Implication({1}, {4}), Implication({5}, {2, 3, 4}), Implication({2}, {4}), Implication({4}, {2})}
K = {Implication({4}, {2})}

In [15]:
ExploringFaultyData(K, L, M)

Значение M - [0, 1, 2, 3, 4, 5]
Значение K - {{4}->{2}}
Значение L - {{5}->{2, 3, 4}, {4}->{2}, {2}->{4}, {1}->{4}}
Значение Pi - set()
Контекст Li - []
NextClosure - {5}
Выбранное Q - {2, 3, 4, 5}
Есть ли контрпример для ['m6']->['m3', 'm4', 'm5', 'm6']
False
{{4}->{2}, {5}->{2, 3, 4, 5}}
NextClosure - {2}
Выбранное Q - {2, 4}
Есть ли контрпример для ['m3']->['m3', 'm5']
[0, 0, 1, 0, 0, 0]
Контекст Li - [[0, 0, 1, 0, 0, 0]]
Значение Li - {{5}->{2, 3, 4}, {4}->{2}, {1}->{4}}
NextClosure - {1}
Выбранное Q - {1, 2, 4}
Есть ли контрпример для ['m2']->['m2', 'm3', 'm5']
[0, 1, 0, 0, 0, 0]
Контекст Li - [[0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0]]
Значение Li - {{5}->{2, 3, 4}, {4}->{2}}
NextClosure - -1
{{4}->{2}, {5}->{2, 3, 4, 5}}
{{5}->{2, 3, 4}, {4}->{2}}
